In [5]:
from flask import Flask, Response, render_template, jsonify
from flask_socketio import SocketIO
import cv2
import dlib
import atexit
import datetime
import os
import time
import threading
from imutils import face_utils
from scipy.spatial import distance
import numpy as np
import tensorflow as tf

app = Flask(__name__)
socketio = SocketIO(app)
model = tf.keras.models.load_model('resnet50_seatbelt_model_checkpoint_epoch_12.h5')

# Initialize video capture and models
video_capture = cv2.VideoCapture(0)  # Use the default camera
face_model = dlib.get_frontal_face_detector()
landmark_model = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# File to keep track of trips and alerts
TRIP_COUNT_FILE = 'trip_count.txt'
ALERTS_FILE = 'alerts.txt'
RECORDINGS_FOLDER = 'recordings'

# Global flag to indicate if the camera is active
camera_active = True
video_writer = None  # To hold the VideoWriter object

# Initialize trip counter
def initialize_trip_counter():
    if not os.path.exists(TRIP_COUNT_FILE):
        with open(TRIP_COUNT_FILE, 'w') as file:
            file.write('0')  # Start from trip 0

# Function to get the current trip number
def get_trip_number():
    with open(TRIP_COUNT_FILE, 'r') as file:
        trip_number = int(file.read().strip())  # Read the current trip number
    return trip_number
def predict_seatbelt(frame, target_size=(120, 160), threshold=0.5):
    # Resize the image to match the model's input shape
    image = cv2.resize(frame, target_size)
    
    # Convert the image to a floating-point array and scale pixel values to [0, 1]
    image = image.astype('float32') / 255.0
    
    # Add a batch dimension
    input_image = np.expand_dims(image, axis=0)

    # Make a prediction
    predictions = model.predict(input_image)

    # Print the predictions
    print("Predictions:", predictions)

    if predictions[0,0]>predictions[0,1]:
       return "person is wearing a seatbelt"
    else:
       return "person is not wearing a seatbelt"
def increment_trip_number():
    trip_number = get_trip_number() + 1  # Increment trip number
    with open(TRIP_COUNT_FILE, 'w') as file:
        file.write(str(trip_number))  # Save the updated trip number
    return trip_number

# Function to save alerts to a text file
def save_alert_to_file(message, timestamp):
    with open(ALERTS_FILE, 'a') as file:
        file.write(f"{timestamp}: {message}\n")

# Function to log the trip start time
def log_trip_start(trip_number):
    start_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(ALERTS_FILE, 'a') as file:
        file.write(f"Trip {trip_number+1} started at: {start_time}\n")

# Function to log the trip end time
def log_trip_end(trip_number):
    end_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(ALERTS_FILE, 'a') as file:
        file.write(f"Trip {trip_number+1} ended at: {end_time}\n")

# Thresholds for yawn detection
yawn_min_thresh = 7
yawn_max_thresh = 20

# Drowsiness detection thresholds
EYE_ASPECT_RATIO_THRESHOLD = 0.25
EYE_ASPECT_RATIO_CONSEC_FRAMES = 30
COUNTER = 0
drowsiness_detected = False
drowsiness_count = 0

# Extract indexes of facial landmarks for the left and right eye
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

# Function to calculate eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2 * C)
    return ear

# Function to calculate mouth aspect ratio (MAR)
def mouth_aspect_ratio(top_lip_point, bottom_lip_point, left_corner, right_corner):
    vertical_distance = bottom_lip_point[1] - top_lip_point[1]
    horizontal_distance = right_corner[0] - left_corner[0]
    mar = vertical_distance / horizontal_distance
    return mar

# Variables for yawning detection
yawn_detected = False
yawn_count = 0
mouth_aspect_ratios = []  # List to store MAR over frames for smoothing

# Generate frames for video feed
def generate_frames():
    yawn_count = 0
    drowsiness_count = 0
    global camera_active, video_writer
    trip_number = get_trip_number()  # Get the current trip number
    log_trip_start(trip_number)  # Log the trip start time

    # Create the recordings directory if it doesn't exist
    if not os.path.exists(RECORDINGS_FOLDER):
        os.makedirs(RECORDINGS_FOLDER)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    video_file_path = os.path.join(RECORDINGS_FOLDER, f'trip_{trip_number+1}.mp4')
    video_writer = cv2.VideoWriter(video_file_path, fourcc, 20.0, (640, 480))  # Change frame size as needed
    
    last_frame_time = time.time()
    while camera_active:
        success, frame = video_capture.read()
        if not success:
            break
        current_time = time.time()
        if current_time - last_frame_time >= 10:
            last_frame_time = current_time  # Update the last frame time
            
            # Predict seatbelt status
            seatbelt_alert = predict_seatbelt(frame)
            print(seatbelt_alert)  # Call the prediction on the current frame

            # Emit seatbelt alert if the person is not wearing a seatbelt
            if seatbelt_alert == "person is not wearing a seatbelt":
                socketio.emit('alert', {'message': seatbelt_alert})  # Send the seatbelt alert to the frontend

        img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_model(img_gray)

        for face in faces:
            shapes = landmark_model(img_gray, face)
            shape = face_utils.shape_to_np(shapes)

            # Yawn Detection (MAR)
            top_lip_point = shape[51]  # Top midpoint of the upper lip
            bottom_lip_point = shape[57]  # Bottom midpoint of the lower lip
            left_corner = shape[48]
            right_corner = shape[54]

            mar = mouth_aspect_ratio(top_lip_point, bottom_lip_point, left_corner, right_corner)
            mouth_aspect_ratios.append(mar)

            if len(mouth_aspect_ratios) > 10:  # Only keep the last 10 values for smoothing
                mouth_aspect_ratios.pop(0)
            avg_mar = np.mean(mouth_aspect_ratios)

            if avg_mar >= 0.5:  # Threshold for yawning
                if not yawn_detected:
                    yawn_detected = True
                    yawn_count += 1
            else:
                yawn_detected = False

            # Drowsiness Detection (EAR)
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEyeAspectRatio = eye_aspect_ratio(leftEye)
            rightEyeAspectRatio = eye_aspect_ratio(rightEye)
            ear = (leftEyeAspectRatio + rightEyeAspectRatio) / 2

            if ear < EYE_ASPECT_RATIO_THRESHOLD:
                COUNTER += 1
                if COUNTER >= EYE_ASPECT_RATIO_CONSEC_FRAMES:
                    if not drowsiness_detected:
                        drowsiness_detected = True
                        drowsiness_count += 1
                    cv2.putText(frame, "Drowsy!", (150, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
            else:
                COUNTER = 0
                drowsiness_detected = False

            # Send alert if counts exceed 2
            if yawn_count > 2 or drowsiness_count > 2:
                socketio.emit('alert', {'message': 'You are too sleepy!'})
                yawn_count = 0
                drowsiness_count = 0

            # Write the frame to the video file
            video_writer.write(frame)

            # Convert the frame to JPEG format
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()

            # Yield the frame as a byte-stream
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    # If the loop breaks, log the trip end
    log_trip_end(trip_number)  # Log the trip end


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

# Route to receive alerts
@app.route('/send_alert/<message>')
def send_alert(message):
    if camera_active:
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        save_alert_to_file(message, timestamp)
        return "Alert received!", 200
    else:
        return "Camera is closed. Alert not saved.", 400

# Route to handle termination
@app.route('/terminate', methods=['POST'])
def terminate():
    global camera_active, video_writer
    camera_active = False  # Set camera active to false
    release_camera()  # Release the camera resources
    increment_trip_number()  # Increment trip number on termination
    return jsonify({"status": "Terminated"})

# Function to release camera and writer resources
def release_camera():
    global video_capture, video_writer
    if video_capture.isOpened():
        video_capture.release()
    if video_writer is not None:
        video_writer.release()

# Ensure resources are released when the program exits
atexit.register(release_camera)

if __name__ == '__main__':
    initialize_trip_counter()
    socketio.run(app, debug=False,allow_unsafe_werkzeug=True)

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Oct/2024 22:23:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2024 22:23:05] "GET /static/tropical-10201.gif HTTP/1.1" 304 -
127.0.0.1 - - [18/Oct/2024 22:23:05] "GET /socket.io/?EIO=4&transport=polling&t=PAWZ0jh HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2024 22:23:05] "GET /drum-bass-243588.mp3 HTTP/1.1" 404 -
127.0.0.1 - - [18/Oct/2024 22:23:05] "POST /socket.io/?EIO=4&transport=polling&t=PAWZ0kn&sid=MPfEkrxdztfVaXHLAAAA HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2024 22:23:05] "GET /socket.io/?EIO=4&transport=polling&t=PAWZ0ko&sid=MPfEkrxdztfVaXHLAAAA HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2024 22:23:05] "GET /socket.io/?EIO=4&transport=polling&t=PAWZ0os&sid=MPfEkrxdztfVaXHLAAAA HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2024 22:23:06] "GET /video_feed HTTP/1.1" 200 -


Predictions: [[0.0133679  0.98663217]]
person is not wearing a seatbelt
Predictions: [[5.910841e-05 9.999409e-01]]
person is not wearing a seatbelt


127.0.0.1 - - [18/Oct/2024 22:23:32] "GET /send_alert/You%20are%20too%20sleepy! HTTP/1.1" 200 -


message sent
Predictions: [[4.0529931e-06 9.9999595e-01]]
person is not wearing a seatbelt


127.0.0.1 - - [18/Oct/2024 22:23:44] "POST /terminate HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2024 22:23:46] "GET /socket.io/?EIO=4&transport=websocket&sid=MPfEkrxdztfVaXHLAAAA HTTP/1.1" 200 -
